# 자동으로 데이터 전처리 및 모델 학습을 위한 pycaret 설치하기

- [pycaret](https://pycaret.org/)은 low-code로 machine learning을 구현할 수 있도록 도와줍니다

- pycaret를 사용하기 위해 설치를 먼저 진행해줍니다

In [ ]:
!pip install pycaret

  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
from pycaret.classification import setup, compare_models, plot_model, predict_model

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 암호화폐 데이터 가져오기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/coin_data/days/KRW-BTC.csv")

# 상승 및 하락 라벨링 하기

In [ ]:
def assign_bull(x):
    if x < 0:
        return 1
    else:
        return 0

def assign_bear(x):
    if x > 0:
        return 1
    else:
        return 0

df['bull'] = df['close'].diff().apply(assign_bull)

df['bear'] = df['close'].diff().apply(assign_bear)


,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate,bull,bear
0,KRW-BTC,2017-09-25T09:00:00,4201000.0,4333000.0,4175000.0,4322000.0,5.602146e+08,132.484755,0.028803,0,0
1,KRW-BTC,2017-09-26T09:00:00,4317000.0,4418000.0,4311000.0,4321000.0,9.950724e+07,22.788340,-0.000231,1,0
2,KRW-BTC,2017-09-27T09:00:00,4322000.0,4677000.0,4318000.0,4657000.0,1.448276e+08,32.269662,0.077760,0,1
3,KRW-BTC,2017-09-28T09:00:00,4657000.0,4772000.0,4519000.0,4586000.0,3.721860e+08,80.588243,-0.015246,1,0
4,KRW-BTC,2017-09-29T09:00:00,4586000.0,4709000.0,4476000.0,4657000.0,2.724558e+08,59.352373,0.015482,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2305,KRW-BTC,2024-01-17T09:00:00,59218000.0,59300000.0,58269000.0,58776000.0,1.666847e+11,2837.334490,-0.007095,1,0
2306,KRW-BTC,2024-01-18T09:00:00,58776000.0,58900000.0,56754000.0,57284000.0,2.472746e+11,4268.883840,-0.025385,1,0
2307,KRW-BTC,2024-01-19T09:00:00,57281000.0,58180000.0,55935000.0,57739000.0,3.199353e+11,5634.881771,0.007943,0,1
2308,KRW-BTC,2024-01-20T09:00:00,57738000.0,57796000.0,57242000.0,57517000.0,1.224661e+11,2127.464542,-0.003845,1,0


# 보조지표 추가하기

In [ ]:
def get_ma_7(df, column):
    ma_7 = df[column].rolling(window=7).mean()
    return ma_7

df["close_ma_7"] = get_ma_7(df, "close")
df["change_rate_ma_7"] = get_ma_7(df, "change_rate")

# 인덱스, 문제 데이터, 정답 데이터 나눠주기

In [ ]:
index_df = df.iloc[:, :2]
X_df = df.drop(columns=list(index_df.columns)  + ["change_rate", "bull", "bear"])
Y_bull_df = df[["bull"]]
Y_bear_df = df[["bear"]]

# 비트코인 데이터의 단위를 작게 만들어주기

In [ ]:
X_df.iloc[:, :-1] = np.log1p(X_df.iloc[:, :-1])

# 이동평균선 적용하느라 발생된 결측치 제외하기

In [ ]:
XY_bull_df = pd.concat([X_df, Y_bull_df], axis=1).iloc[6:]
XY_bear_df = pd.concat([X_df, Y_bear_df], axis=1).iloc[6:]

# 데이터 전처리 및 여러 모델별로 성능 비교해보기

In [ ]:
test_size = 30
def get_model(df, target, test_size):
    exp1 = setup(
        df.iloc[:-test_size],
        target=target,
        train_size=0.9,
        verbose=False
        )
    return compare_models()


# 상승 예측 여러 모델의 성능 비교

In [ ]:
bull_model = get_model(XY_bull_df, "bull", test_size)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8528,0.9469,0.7742,0.9044,0.8337,0.7032,0.7106,0.0300
ridge,Ridge Classifier,0.8499,0.0000,0.7436,0.9279,0.8253,0.6965,0.7100,0.0270
qda,Quadratic Discriminant Analysis,0.7977,0.9212,0.6628,0.8887,0.7551,0.5899,0.6116,0.0630
lightgbm,Light Gradient Boosting Machine,0.7568,0.8530,0.7395,0.7488,0.7433,0.5123,0.5134,0.6830
xgboost,Extreme Gradient Boosting,0.7485,0.8274,0.7283,0.7421,0.7342,0.4956,0.4968,0.2180
et,Extra Trees Classifier,0.6735,0.7281,0.6434,0.6620,0.6519,0.3447,0.3453,0.3090
gbc,Gradient Boosting Classifier,0.6662,0.7347,0.6067,0.6653,0.6337,0.3283,0.3302,0.5080
ada,Ada Boost Classifier,0.6311,0.6606,0.5668,0.6275,0.5940,0.2577,0.2600,0.4030
rf,Random Forest Classifier,0.6150,0.6627,0.5730,0.6010,0.5859,0.2267,0.2274,0.7330
dt,Decision Tree Classifier,0.5765,0.5768,0.5832,0.5525,0.5668,0.1532,0.1538,0.0490


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

# 학습된 상승 예측 모델로 예측해보기

In [ ]:
predictions = predict_model(bull_model, data=XY_bull_df[-test_size:])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.9000,0.9910,1.0000,0.8500,0.9189,0.7907,0.8086


# 실제 상승 여부와 예측값 비교해보기

In [ ]:
predictions

,open,high,low,close,trade_price,trade_volume,close_ma_7,change_rate_ma_7,bull,prediction_label,prediction_score
2280,59532000.0,59700000.0,59000000.0,59250000.0,1.389391e+11,2340.108398,58613856.0,0.003334,1,1,0.6892
2281,59249000.0,59600000.0,58000000.0,58002000.0,2.200170e+11,3729.333496,58798428.0,0.003346,1,1,0.8813
2282,58002000.0,58659000.0,57464000.0,58042000.0,2.721329e+11,4687.208984,58785856.0,-0.000133,0,1,0.5263
2283,58042000.0,58140000.0,56349000.0,57032000.0,2.708228e+11,4735.230469,58669716.0,-0.001918,1,1,0.8159
2284,57032000.0,58212000.0,56349000.0,57920000.0,2.270096e+11,3965.842773,58465284.0,-0.003413,0,0,0.7701
2285,57920000.0,58468000.0,56700000.0,56907000.0,2.269573e+11,3958.816650,58097856.0,-0.006220,1,1,0.8515
2286,56904000.0,57300000.0,56000000.0,56450000.0,2.419572e+11,4273.270508,57657572.0,-0.007495,1,1,0.7117
2287,56450000.0,57150000.0,56101000.0,56639000.0,1.301367e+11,2301.724854,57284572.0,-0.006340,0,1,0.5088
2288,56639000.0,57437000.0,56452000.0,57047000.0,1.315195e+11,2308.383057,57148144.0,-0.002301,0,0,0.5709
2289,57045000.0,58847000.0,57045000.0,58839000.0,1.824912e+11,3165.321045,57262000.0,0.002088,0,0,0.9115


# 하락 예측 여러 모델들의 성능 비교

In [ ]:
bear_model = get_model(XY_bear_df, "bear", test_size)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8665,0.9524,0.9363,0.8309,0.8799,0.7309,0.7387,0.0310
ridge,Ridge Classifier,0.8611,0.0000,0.9513,0.8151,0.8774,0.7196,0.7323,0.0260
lightgbm,Light Gradient Boosting Machine,0.7778,0.8611,0.7938,0.7829,0.7880,0.5545,0.5551,0.3370
qda,Quadratic Discriminant Analysis,0.7661,0.9206,0.9626,0.7022,0.8114,0.5237,0.5710,0.0290
xgboost,Extreme Gradient Boosting,0.7563,0.8372,0.7826,0.7575,0.7693,0.5113,0.5124,0.1360
gbc,Gradient Boosting Classifier,0.6720,0.7426,0.7039,0.6782,0.6902,0.3420,0.3430,0.5070
et,Extra Trees Classifier,0.6720,0.7328,0.6870,0.6843,0.6847,0.3429,0.3439,0.5750
rf,Random Forest Classifier,0.6228,0.6747,0.6504,0.6342,0.6417,0.2435,0.2440,0.8610
ada,Ada Boost Classifier,0.6194,0.6525,0.6655,0.6271,0.6447,0.2356,0.2370,0.2070
dt,Decision Tree Classifier,0.5682,0.5680,0.5736,0.5871,0.5797,0.1358,0.1362,0.0810


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

# 학습된 하락 예측 모델로 예측해보기

In [ ]:
predictions = predict_model(bear_model, data=XY_bear_df[-test_size:])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.9000,0.9864,0.7692,1.0000,0.8696,0.7907,0.8086


# 실제 하락 여부와 예측값 비교해보기

In [ ]:
predictions

,open,high,low,close,trade_price,trade_volume,close_ma_7,change_rate_ma_7,bear,prediction_label,prediction_score
2280,59532000.0,59700000.0,59000000.0,59250000.0,1.389391e+11,2340.108398,58613856.0,0.003334,0,0,0.6840
2281,59249000.0,59600000.0,58000000.0,58002000.0,2.200170e+11,3729.333496,58798428.0,0.003346,0,0,0.8850
2282,58002000.0,58659000.0,57464000.0,58042000.0,2.721329e+11,4687.208984,58785856.0,-0.000133,1,0,0.5411
2283,58042000.0,58140000.0,56349000.0,57032000.0,2.708228e+11,4735.230469,58669716.0,-0.001918,0,0,0.8277
2284,57032000.0,58212000.0,56349000.0,57920000.0,2.270096e+11,3965.842773,58465284.0,-0.003413,1,1,0.7620
2285,57920000.0,58468000.0,56700000.0,56907000.0,2.269573e+11,3958.816650,58097856.0,-0.006220,0,0,0.8536
2286,56904000.0,57300000.0,56000000.0,56450000.0,2.419572e+11,4273.270508,57657572.0,-0.007495,0,0,0.7194
2287,56450000.0,57150000.0,56101000.0,56639000.0,1.301367e+11,2301.724854,57284572.0,-0.006340,1,0,0.5142
2288,56639000.0,57437000.0,56452000.0,57047000.0,1.315195e+11,2308.383057,57148144.0,-0.002301,1,1,0.5681
2289,57045000.0,58847000.0,57045000.0,58839000.0,1.824912e+11,3165.321045,57262000.0,0.002088,1,1,0.9157
